In [1]:
%load_ext autoreload
%load_ext autotime
%autoreload 2
%matplotlib inline

In [2]:
import multiprocessing
from multiprocessing import Pool
from joblib import Parallel, delayed

time: 61.1 ms


In [3]:
multiprocessing.cpu_count()

16

time: 2.47 ms


In [ ]:
import time

def basic_func(x):
    if x == 0:
        return 'zero'
    elif x%2 == 0:
        return 'even'
    else:
        return 'odd'
    
starttime = time.time()
for i in range(0,10):
    y = i*i
    time.sleep(2)
    print('{} squared results in a/an {} number'.format(i, basic_func(y)))
    
print('That took {} seconds'.format(time.time() - starttime))

In [ ]:
import time
import multiprocessing 

def basic_func(x):
    if x == 0:
        return 'zero'
    elif x%2 == 0:
        return 'even'
    else:
        return 'odd'

def multiprocessing_func(x):
    y = x*x
    time.sleep(2)
    print('{} squared results in a/an {} number'.format(x, basic_func(y)))
    
if __name__ == '__main__':
    starttime = time.time()
    processes = []
    for i in range(0,30):
        p = multiprocessing.Process(target=multiprocessing_func, args=(i,))
        processes.append(p)
        p.start()
        
    for process in processes:
        process.join()
        
    print('That took {} seconds'.format(time.time() - starttime))

In [ ]:
import time
import multiprocessing 

def basic_func(x):
    if x == 0:
        return 'zero'
    elif x%2 == 0:
        return 'even'
    else:
        return 'odd'

def multiprocessing_func(x):
    y = x*x
    time.sleep(2)
    print('{} squared results in a/an {} number'.format(x, basic_func(y)))
    
if __name__ == '__main__':
    
    starttime = time.time()
    pool = multiprocessing.Pool(16)
    pool.map(multiprocessing_func, range(0,30))
    pool.close()
    print('That took {} seconds'.format(time.time() - starttime))

In [4]:
import os
import sys
import logging

sys.path.insert(0, '.')
sys.path.insert(0, '..')

import math
import numpy as np
import pandas as pd
import cv2 as cv
from glob import glob

logging.basicConfig(format='%(name)s - %(levelname)s - %(asctime)s - %(filename)s - %(lineno)d - %(message)s', level=logging.DEBUG)

time: 1.4 s


In [5]:
# configs
BASE_FOLDER = '..'
DATA_FOLDER = 'data'
TRAIN_FRAMES = 'train'
VALID_FRAMES = 'valid'
TEST_FRAMES = 'test'
TRAIN_VIDEOS = 'train_videos'
CHUNK_NAME = 'train_sample_videos'
TEST_VIDEOS = 'test_videos'
METADATA = 'metadata'

time: 1.09 ms


In [56]:
# ========== creating directories
logging.info("Creating directories (if needed)")
data_dir = os.path.join(BASE_FOLDER, DATA_FOLDER)

# train frames folder
train_dir = os.path.join(data_dir, TRAIN_FRAMES)
if not os.path.isdir(train_dir):
    os.mkdir(train_dir)

if not os.path.isdir(os.path.join(train_dir, 'REAL')):
    os.mkdir(os.path.join(train_dir, 'REAL'))

if not os.path.isdir(os.path.join(train_dir, 'FAKE')):
    os.mkdir(os.path.join(train_dir, 'FAKE'))

# valid frames folder
valid_dir = os.path.join(data_dir, VALID_FRAMES)
if not os.path.isdir(valid_dir):
    os.mkdir(valid_dir)

if not os.path.isdir(os.path.join(valid_dir, 'REAL')):
    os.mkdir(os.path.join(valid_dir, 'REAL'))

if not os.path.isdir(os.path.join(valid_dir, 'FAKE')):
    os.mkdir(os.path.join(valid_dir, 'FAKE'))

# test frames folder
test_dir = os.path.join(data_dir, TEST_FRAMES)
if not os.path.isdir(test_dir):
    os.mkdir(test_dir)

root - INFO - 2020-03-26 14:43:18,991 - <ipython-input-56-facaac388ece> - 2 - Creating directories (if needed)


time: 5.16 ms


In [39]:
# ========== getting a count of frames in each folder
logging.info(f"Total train videos: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, TRAIN_VIDEOS, CHUNK_NAME)))}")
logging.info(f"Total test videos: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, TEST_VIDEOS)))}")

root - INFO - 2020-03-26 13:18:03,249 - <ipython-input-39-fd64b2882b08> - 2 - Total train videos: 50
root - INFO - 2020-03-26 13:18:03,250 - <ipython-input-39-fd64b2882b08> - 3 - Total test videos: 20


time: 2.2 ms


In [40]:
# ========== functions
def get_metadata(jsonpath):
    metadf = pd.DataFrame()
    json_pattern = os.path.join(jsonpath, '*.json')
    flist = glob(json_pattern)
    for f in flist:
        df = pd.read_json(f)
        df = df.T
        metadf = metadf.append(df, ignore_index = False)
    return metadf

def get_frames(cap, name, dirname, metadata, test=False):
    frameRate = cap.get(5) #to get the frame rate
    if not test:
        label = metadata.loc[metadata.index == name, "label"].values[0]
    count = 0
    logging.info(f"capturing frames of video file: {name}")
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            logging.info("frames couldn't be captured!")
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a different folder
            image = frame
            count += 1
            logging.info(f"frame number: {count}")
            fn = os.path.splitext(name)[0] + f"_frame{count}.jpg"
            if test:
                file_path = os.path.join(dirname, fn)
            else:
                file_path = os.path.join(dirname, label, fn)
            logging.info(f"storing frame in: {file_path}")
            cv.imwrite(file_path, image)
    cap.release()
    logging.info("="*50)

def walkdir(dirpath):
    for root, dirs, files in os.walk(dirpath):
        for name in files:
            if name != 'metadata.json':
                yield os.path.abspath(os.path.join(dirpath, name)), name

time: 4.05 ms


In [10]:
global test
test = False

time: 2.51 ms


In [11]:
def get_video_path(datadir):
    if test:
        video_path = os.path.join(datadir, TEST_VIDEOS)
        dirname = os.path.join(datadir, TEST_FRAMES)
    else:
        video_path = os.path.join(datadir, TRAIN_VIDEOS, CHUNK_NAME)
    return video_path

time: 868 µs


In [12]:
def main(video_path):
    cnt = 1
    for path, name in walkdir(video_path):
        cap = cv.VideoCapture(path)
        if not test:
            if cnt <= traincount:
                dirname = os.path.join(BASE_FOLDER, DATA_FOLDER, TRAIN_FRAMES)
            else:
                dirname = os.path.join(BASE_FOLDER, DATA_FOLDER, VALID_FRAMES)
        else:
            dirname = os.path.join(BASE_FOLDER, DATA_FOLDER, TEST_FRAMES)
        get_frames(cap, name, dirname, metadata, test)
        cnt += 1

time: 1.34 ms


In [70]:
datadir = os.path.join(BASE_FOLDER, DATA_FOLDER)
video_path = get_video_path(datadir)
videos_list = glob(os.path.join(video_path, '*.mp4'))

time: 1.18 ms


In [74]:
def main_parallel(cnt, video_path, testtmp):
    name = video_path.split('/')[-1]
    cap = cv.VideoCapture(video_path)
    cnt += 1
    if not testtmp:
        if cnt <= traincount:
            dirname = os.path.join(BASE_FOLDER, DATA_FOLDER, TRAIN_FRAMES)
        else:
            dirname = os.path.join(BASE_FOLDER, DATA_FOLDER, VALID_FRAMES)
    else:
        dirname = os.path.join(BASE_FOLDER, DATA_FOLDER, TEST_FRAMES)
    get_frames(cap, name, dirname, metadata, testtmp)

time: 1.53 ms


In [50]:
processes = []
for cnt, video_path in enumerate(videos_list):
    p = multiprocessing.Process(target=main_parallel, args=(cnt, video_path,))
    processes.append(p)
    p.start()
for process in processes:
    process.join()

root - INFO - 2020-03-26 13:47:44,671 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: eqvuznuwsa.mp4
root - INFO - 2020-03-26 13:47:44,680 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: emgjphonqb.mp4
root - INFO - 2020-03-26 13:47:44,698 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: ensyyivobf.mp4
root - INFO - 2020-03-26 13:47:44,702 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: ejkqesyvam.mp4
root - INFO - 2020-03-26 13:47:44,702 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: eudeqjhdfd.mp4
root - INFO - 2020-03-26 13:47:44,704 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: eqjscdagiv.mp4
root - INFO - 2020-03-26 13:47:44,726 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: esgftaficx.mp4
root - INFO - 2020-03-26 13:47:44,779 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: esnntzz

root - INFO - 2020-03-26 13:47:49,543 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 1
root - INFO - 2020-03-26 13:47:49,929 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eejswgycjc_frame1.jpg
root - INFO - 2020-03-26 13:47:49,865 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: elvvackpjh.mp4
root - INFO - 2020-03-26 13:47:49,887 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 1
root - INFO - 2020-03-26 13:47:49,732 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 1
root - INFO - 2020-03-26 13:47:50,007 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eepezmygaq_frame1.jpg
root - INFO - 2020-03-26 13:47:50,095 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/etmcruaihe_frame1.jpg
root - INFO - 2020-03-26 13:47:49,969 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 1
root - INFO - 2020-03-26 13:47:50,012 - <ipython-input-40-6cde47138ac0> - 28 - f

root - INFO - 2020-03-26 13:47:57,948 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 1
root - INFO - 2020-03-26 13:47:57,666 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 1
root - INFO - 2020-03-26 13:47:57,964 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/ellavthztb_frame1.jpg
root - INFO - 2020-03-26 13:47:57,984 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esckbnkkvb_frame1.jpg
root - INFO - 2020-03-26 13:47:57,967 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 1
root - INFO - 2020-03-26 13:47:58,005 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/erqgqacbqe_frame1.jpg
root - INFO - 2020-03-26 13:47:58,017 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/erlvuvjsjf_frame1.jpg
root - INFO - 2020-03-26 13:47:57,894 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 1
root - INFO - 2020-03-26 13:47:57,830 - <ipython-input-40-6cde47138a

root - INFO - 2020-03-26 13:48:02,487 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esyrimvzsa_frame3.jpg
root - INFO - 2020-03-26 13:48:02,434 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 3
root - INFO - 2020-03-26 13:48:02,236 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 13:48:02,600 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ejkqesyvam_frame2.jpg
root - INFO - 2020-03-26 13:48:02,512 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eixwxvxbbn_frame2.jpg
root - INFO - 2020-03-26 13:48:02,567 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 13:48:02,603 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ehevsxtecd_frame3.jpg
root - INFO - 2020-03-26 13:48:02,168 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 13:48:02,524 - <ipython-input-40-6cde47138a

root - INFO - 2020-03-26 13:48:04,042 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 13:48:04,178 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/eqnoqyfquo_frame2.jpg
root - INFO - 2020-03-26 13:48:04,191 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 13:48:04,208 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eekozbeafq_frame2.jpg
root - INFO - 2020-03-26 13:48:04,321 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 13:48:04,358 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esyhwdfnxs_frame2.jpg
root - INFO - 2020-03-26 13:48:04,366 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 13:48:04,400 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eivxffliio_frame2.jpg
root - INFO - 2020-03-26 13:48:04,418 - <ipython-input-40-6cde47138a

...

root - INFO - 2020-03-26 13:48:26,683 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 11
root - INFO - 2020-03-26 13:48:26,705 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esyrimvzsa_frame11.jpg
root - INFO - 2020-03-26 13:48:26,757 - <ipython-input-40-6cde47138ac0> - 22 - frames couldn't be captured!
root - INFO - 2020-03-26 13:48:26,802 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 10
root - INFO - 2020-03-26 13:48:26,820 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eoewqcpbgt_frame10.jpg
root - INFO - 2020-03-26 13:48:26,905 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 11
root - INFO - 2020-03-26 13:48:26,930 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 11
root - INFO - 2020-03-26 13:48:26,932 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/epymyyiblu_frame11.jpg
root - INFO - 2020-03-26 13:48:26,934 - <ipython-input-40-6cde47138ac0> - 34 - storing fram

root - INFO - 2020-03-26 13:48:28,371 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 10
root - INFO - 2020-03-26 13:48:28,358 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 10
root - INFO - 2020-03-26 13:48:28,394 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/egghxjjmfg_frame10.jpg
root - INFO - 2020-03-26 13:48:28,394 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esyhwdfnxs_frame10.jpg
root - INFO - 2020-03-26 13:48:28,420 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 13:48:28,431 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 10
root - INFO - 2020-03-26 13:48:28,428 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 10
root - INFO - 2020-03-26 13:48:28,454 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eivxffliio_frame10.jpg
root - INFO - 2020-03-26 13:48:28,457 - <ipython-input-40-6cde47138ac

root - INFO - 2020-03-26 13:48:30,018 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 13:48:30,031 - <ipython-input-40-6cde47138ac0> - 22 - frames couldn't be captured!
root - INFO - 2020-03-26 13:48:30,013 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 11
root - INFO - 2020-03-26 13:48:30,057 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ehieahnhte_frame11.jpg
root - INFO - 2020-03-26 13:48:30,068 - <ipython-input-40-6cde47138ac0> - 22 - frames couldn't be captured!
root - INFO - 2020-03-26 13:48:30,066 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 11
root - INFO - 2020-03-26 13:48:30,099 - <ipython-input-40-6cde47138ac0> - 22 - frames couldn't be captured!
root - INFO - 2020-03-26 13:48:30,100 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 11
root - INFO - 2020-03-26 13:48:30,101 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ehb

root - INFO - 2020-03-26 13:48:30,537 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 13:48:30,541 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 13:48:30,554 - <ipython-input-40-6cde47138ac0> - 22 - frames couldn't be captured!
root - INFO - 2020-03-26 13:48:30,586 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 13:48:30,589 - <ipython-input-40-6cde47138ac0> - 22 - frames couldn't be captured!
root - INFO - 2020-03-26 13:48:30,593 - <ipython-input-40-6cde47138ac0> - 22 - frames couldn't be captured!
root - INFO - 2020-03-26 13:48:30,592 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 13:48:30,599 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 1

time: 46.2 s


In [73]:
print(f"Total REAL train: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, TRAIN_FRAMES, 'REAL')))}")
print(f"Total FAKE train: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, TRAIN_FRAMES, 'FAKE')))}")
print(f"Total REAL valid: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, VALID_FRAMES, 'REAL')))}")
print(f"Total FAKE valid: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, VALID_FRAMES, 'FAKE')))}")

Total REAL train: 66
Total FAKE train: 363
Total REAL valid: 33
Total FAKE valid: 77
time: 5.23 ms


In [72]:
pool = multiprocessing.Pool()
pool.starmap(main_parallel, enumerate(videos_list))
pool.close()

root - INFO - 2020-03-26 16:07:08,972 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: eprybmbpba.mp4
root - INFO - 2020-03-26 16:07:08,972 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: esnntzzajv.mp4
root - INFO - 2020-03-26 16:07:08,977 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: eukvucdetx.mp4
root - INFO - 2020-03-26 16:07:08,979 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: emaalmsonj.mp4
root - INFO - 2020-03-26 16:07:08,978 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: eejswgycjc.mp4
root - INFO - 2020-03-26 16:07:08,987 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: ejkqesyvam.mp4
root - INFO - 2020-03-26 16:07:08,983 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: esgftaficx.mp4
root - INFO - 2020-03-26 16:07:08,985 - <ipython-input-40-6cde47138ac0> - 17 - capturing frames of video file: eqnoqyf

root - INFO - 2020-03-26 16:07:11,625 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 16:07:11,639 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 16:07:11,643 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 16:07:11,645 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/etejaapnxh_frame2.jpg
root - INFO - 2020-03-26 16:07:11,644 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 16:07:11,648 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esnntzzajv_frame2.jpg
root - INFO - 2020-03-26 16:07:11,648 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/eqnoqyfquo_frame2.jpg
root - INFO - 2020-03-26 16:07:11,648 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eejswgycjc_frame2.jpg
root - INFO - 2020-03-26 16:07:11,649 - <ipython-input-40-6cde47138a

root - INFO - 2020-03-26 16:07:13,714 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 4
root - INFO - 2020-03-26 16:07:13,725 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 4
root - INFO - 2020-03-26 16:07:13,726 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/eudeqjhdfd_frame4.jpg
root - INFO - 2020-03-26 16:07:13,728 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esnntzzajv_frame4.jpg
root - INFO - 2020-03-26 16:07:14,276 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 5
root - INFO - 2020-03-26 16:07:14,323 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eprybmbpba_frame5.jpg
root - INFO - 2020-03-26 16:07:14,489 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 5
root - INFO - 2020-03-26 16:07:14,539 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/emaalmsonj_frame5.jpg
root - INFO - 2020-03-26 16:07:14,522 - <ipython-input-40-6cde47138a

root - INFO - 2020-03-26 16:07:16,386 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/emaalmsonj_frame7.jpg
root - INFO - 2020-03-26 16:07:16,415 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 7
root - INFO - 2020-03-26 16:07:16,466 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/elginszwtk_frame7.jpg
root - INFO - 2020-03-26 16:07:16,475 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 7
root - INFO - 2020-03-26 16:07:16,482 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eukvucdetx_frame7.jpg
root - INFO - 2020-03-26 16:07:16,514 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 7
root - INFO - 2020-03-26 16:07:16,518 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 7
root - INFO - 2020-03-26 16:07:16,526 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ejkqesyvam_frame7.jpg
root - INFO - 2020-03-26 16:07:16,529 - <ipython-input-40-6cde47138a

root - INFO - 2020-03-26 16:07:18,398 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 9
root - INFO - 2020-03-26 16:07:18,412 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/ehtdtkmmli_frame9.jpg
root - INFO - 2020-03-26 16:07:18,420 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 9
root - INFO - 2020-03-26 16:07:18,426 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/eqnoqyfquo_frame9.jpg
root - INFO - 2020-03-26 16:07:18,435 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 9
root - INFO - 2020-03-26 16:07:18,430 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 9
root - INFO - 2020-03-26 16:07:18,444 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/emgjphonqb_frame9.jpg
root - INFO - 2020-03-26 16:07:18,447 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eqvuznuwsa_frame9.jpg
root - INFO - 2020-03-26 16:07:18,455 - <ipython-input-40-6cde47138a

...

root - INFO - 2020-03-26 16:07:37,271 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ekhacizpah_frame6.jpg
root - INFO - 2020-03-26 16:07:37,374 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 6
root - INFO - 2020-03-26 16:07:37,385 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ekkdjkirzq_frame6.jpg
root - INFO - 2020-03-26 16:07:37,380 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 6
root - INFO - 2020-03-26 16:07:37,394 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/erqgqacbqe_frame6.jpg
root - INFO - 2020-03-26 16:07:37,512 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 6
root - INFO - 2020-03-26 16:07:37,521 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/valid/FAKE/ehevsxtecd_frame6.jpg
root - INFO - 2020-03-26 16:07:37,517 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 6
root - INFO - 2020-03-26 16:07:37,522 - <ipython-input-40-6cde47138a

root - INFO - 2020-03-26 16:07:39,683 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ekhacizpah_frame8.jpg
root - INFO - 2020-03-26 16:07:39,685 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/valid/FAKE/esyrimvzsa_frame8.jpg
root - INFO - 2020-03-26 16:07:39,697 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 8
root - INFO - 2020-03-26 16:07:39,703 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/valid/REAL/ehccixxzoe_frame8.jpg
root - INFO - 2020-03-26 16:07:39,717 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 8
root - INFO - 2020-03-26 16:07:39,725 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/valid/REAL/erlvuvjsjf_frame8.jpg
root - INFO - 2020-03-26 16:07:39,751 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 8
root - INFO - 2020-03-26 16:07:39,761 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 8
root - INFO - 2020-03-26 16:07:39,765 - <ipython-input-40-6cde47138a

root - INFO - 2020-03-26 16:07:42,006 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/valid/REAL/erlvuvjsjf_frame10.jpg
root - INFO - 2020-03-26 16:07:42,006 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/valid/REAL/ehccixxzoe_frame10.jpg
root - INFO - 2020-03-26 16:07:42,006 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/valid/FAKE/epymyyiblu_frame10.jpg
root - INFO - 2020-03-26 16:07:42,041 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 10
root - INFO - 2020-03-26 16:07:42,048 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/eggbjzxnmg_frame10.jpg
root - INFO - 2020-03-26 16:07:42,061 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 11
root - INFO - 2020-03-26 16:07:42,093 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esyhwdfnxs_frame11.jpg
root - INFO - 2020-03-26 16:07:42,120 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 10
root - INFO - 202

root - INFO - 2020-03-26 16:07:43,203 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 16:07:43,223 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 16:07:43,235 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 16:07:43,253 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 16:07:43,287 - <ipython-input-40-6cde47138ac0> - 22 - frames couldn't be captured!
root - INFO - 2020-03-26 16:07:43,325 - <ipython-input-40-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 16:07:43,330 - <ipython-input-40-6cde47138ac0> - 34 - storing frame in: ../data/valid/FAKE/esckbnkkvb_frame2.jpg
root - INFO - 2020-03-26 16:07:43,347 - <ipython-input-40-6cde47138ac0> - 37 - ==================================================
root - IN

time: 35.9 s


In [53]:
len(videos_list)

49

time: 1.47 ms


In [55]:
train_val_split = 0.8
filescount = len(videos_list)
traincount = int(filescount * train_val_split)

time: 697 µs


In [37]:
# non-parallel
datadir = os.path.join(BASE_FOLDER, DATA_FOLDER)
video_path = get_video_path(datadir)
main(video_path)

root - INFO - 2020-03-26 13:16:36,123 - <ipython-input-35-6cde47138ac0> - 17 - capturing frames of video file: eqvuznuwsa.mp4
root - INFO - 2020-03-26 13:16:36,189 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 1
root - INFO - 2020-03-26 13:16:36,190 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eqvuznuwsa_frame1.jpg
root - INFO - 2020-03-26 13:16:36,393 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 13:16:36,394 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eqvuznuwsa_frame2.jpg
root - INFO - 2020-03-26 13:16:36,561 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 3
root - INFO - 2020-03-26 13:16:36,562 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eqvuznuwsa_frame3.jpg
root - INFO - 2020-03-26 13:16:36,709 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 4
root - INFO - 2020-03-26 13:16:36,709 - <ipython-input-35-6cde47138ac0> - 34 - s

root - INFO - 2020-03-26 13:16:40,483 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 11
root - INFO - 2020-03-26 13:16:40,483 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/emgjphonqb_frame11.jpg
root - INFO - 2020-03-26 13:16:40,522 - <ipython-input-35-6cde47138ac0> - 22 - frames couldn't be captured!
root - INFO - 2020-03-26 13:16:40,539 - <ipython-input-35-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 13:16:40,563 - <ipython-input-35-6cde47138ac0> - 17 - capturing frames of video file: ensyyivobf.mp4
root - INFO - 2020-03-26 13:16:40,619 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 1
root - INFO - 2020-03-26 13:16:40,620 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ensyyivobf_frame1.jpg
root - INFO - 2020-03-26 13:16:40,743 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 2
root - INFO - 2020-03-26 13:16:40,743 - <ipython-input-35-6cde47138ac0> -

root - INFO - 2020-03-26 13:16:44,032 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 9
root - INFO - 2020-03-26 13:16:44,032 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ejkqesyvam_frame9.jpg
root - INFO - 2020-03-26 13:16:44,146 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 10
root - INFO - 2020-03-26 13:16:44,147 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ejkqesyvam_frame10.jpg
root - INFO - 2020-03-26 13:16:44,253 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 11
root - INFO - 2020-03-26 13:16:44,254 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/ejkqesyvam_frame11.jpg
root - INFO - 2020-03-26 13:16:44,290 - <ipython-input-35-6cde47138ac0> - 22 - frames couldn't be captured!
root - INFO - 2020-03-26 13:16:44,305 - <ipython-input-35-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 13:16:44,326 - <ipython-input-35-6

root - INFO - 2020-03-26 13:16:47,521 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 7
root - INFO - 2020-03-26 13:16:47,522 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eprybmbpba_frame7.jpg
root - INFO - 2020-03-26 13:16:47,633 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 8
root - INFO - 2020-03-26 13:16:47,634 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eprybmbpba_frame8.jpg
root - INFO - 2020-03-26 13:16:47,741 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 9
root - INFO - 2020-03-26 13:16:47,742 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eprybmbpba_frame9.jpg
root - INFO - 2020-03-26 13:16:47,853 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 10
root - INFO - 2020-03-26 13:16:47,853 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/eprybmbpba_frame10.jpg
root - INFO - 2020-03-26 13:16:47,958 - <ipython-input-35-6cde4713

root - INFO - 2020-03-26 13:16:51,076 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 5
root - INFO - 2020-03-26 13:16:51,077 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/ehtdtkmmli_frame5.jpg
root - INFO - 2020-03-26 13:16:51,186 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 6
root - INFO - 2020-03-26 13:16:51,186 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/ehtdtkmmli_frame6.jpg
root - INFO - 2020-03-26 13:16:51,320 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 7
root - INFO - 2020-03-26 13:16:51,321 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/ehtdtkmmli_frame7.jpg
root - INFO - 2020-03-26 13:16:51,427 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 8
root - INFO - 2020-03-26 13:16:51,427 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/ehtdtkmmli_frame8.jpg
root - INFO - 2020-03-26 13:16:51,525 - <ipython-input-35-6cde47138a

...

root - INFO - 2020-03-26 13:17:26,767 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 9
root - INFO - 2020-03-26 13:17:26,768 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/ehccixxzoe_frame9.jpg
root - INFO - 2020-03-26 13:17:26,888 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 10
root - INFO - 2020-03-26 13:17:26,888 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/ehccixxzoe_frame10.jpg
root - INFO - 2020-03-26 13:17:26,994 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 11
root - INFO - 2020-03-26 13:17:26,994 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/REAL/ehccixxzoe_frame11.jpg
root - INFO - 2020-03-26 13:17:27,026 - <ipython-input-35-6cde47138ac0> - 22 - frames couldn't be captured!
root - INFO - 2020-03-26 13:17:27,039 - <ipython-input-35-6cde47138ac0> - 37 - ==================================================
root - INFO - 2020-03-26 13:17:27,058 - <ipython-input-35-6

root - INFO - 2020-03-26 13:17:30,299 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 7
root - INFO - 2020-03-26 13:17:30,300 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esyrimvzsa_frame7.jpg
root - INFO - 2020-03-26 13:17:30,411 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 8
root - INFO - 2020-03-26 13:17:30,412 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esyrimvzsa_frame8.jpg
root - INFO - 2020-03-26 13:17:30,527 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 9
root - INFO - 2020-03-26 13:17:30,527 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esyrimvzsa_frame9.jpg
root - INFO - 2020-03-26 13:17:30,635 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 10
root - INFO - 2020-03-26 13:17:30,636 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esyrimvzsa_frame10.jpg
root - INFO - 2020-03-26 13:17:30,748 - <ipython-input-35-6cde4713

root - INFO - 2020-03-26 13:17:33,824 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 5
root - INFO - 2020-03-26 13:17:33,825 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/epymyyiblu_frame5.jpg
root - INFO - 2020-03-26 13:17:33,933 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 6
root - INFO - 2020-03-26 13:17:33,933 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/epymyyiblu_frame6.jpg
root - INFO - 2020-03-26 13:17:34,048 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 7
root - INFO - 2020-03-26 13:17:34,048 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/epymyyiblu_frame7.jpg
root - INFO - 2020-03-26 13:17:34,156 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 8
root - INFO - 2020-03-26 13:17:34,156 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/epymyyiblu_frame8.jpg
root - INFO - 2020-03-26 13:17:34,265 - <ipython-input-35-6cde47138a

root - INFO - 2020-03-26 13:17:37,270 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 3
root - INFO - 2020-03-26 13:17:37,270 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esckbnkkvb_frame3.jpg
root - INFO - 2020-03-26 13:17:37,380 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 4
root - INFO - 2020-03-26 13:17:37,380 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esckbnkkvb_frame4.jpg
root - INFO - 2020-03-26 13:17:37,489 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 5
root - INFO - 2020-03-26 13:17:37,489 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esckbnkkvb_frame5.jpg
root - INFO - 2020-03-26 13:17:37,599 - <ipython-input-35-6cde47138ac0> - 28 - frame number: 6
root - INFO - 2020-03-26 13:17:37,600 - <ipython-input-35-6cde47138ac0> - 34 - storing frame in: ../data/train/FAKE/esckbnkkvb_frame6.jpg
root - INFO - 2020-03-26 13:17:37,706 - <ipython-input-35-6cde47138a

time: 1min 2s


In [76]:
import argparse

parser = argparse.ArgumentParser(description='Short sample app')

parser.add_argument('-a', action="store_true", default=False)
parser.add_argument('-b', action="store", dest="b")
parser.add_argument('-c', action="store", dest="c", type=int)

print(parser.parse_args(['-a', '-bval', '-c', '3']))

Namespace(a=True, b='val', c=3)
time: 3.71 ms


In [82]:
args = parser.parse_args(['-b', 'val', '-c', '3'])
args.a

False

time: 1.71 ms


In [84]:
args.c

3

time: 1.3 ms
